# Make a set of Amazon Personalize campaigns
Now that the data has been generated, the datasets have been created, and the
data has been imported, now it is time to create Personalize solutions, solution
versions, and campaigns.

In [147]:
import json
import boto3
import time

region      = '<your-region>'
account_num = '<your-account>'
dataset_group_name = 'car-dg12'

dataset_group_arn = 'arn:aws:personalize:{}:{}:dataset-group/{}'.format(region, 
                                                                        account_num, 
                                                                        dataset_group_name)

MAX_WAIT_TIME = 60*60 # 1 hour
SLEEP_TIME    = 60    # 1 minute

In [148]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Function to create a new campaign for a specific recipe
This function creates a new campaign in our dataset group based on a specific
Personalize recipe.

In [183]:
def solution_exists(solution_arn):
    _exists = False
    try:
        _resp   = personalize.describe_solution(solutionArn = solution_arn)
        _exists = True
    except Exception as e:
        print('Error calling describe_solution: {}'.format(e))
        pass
        
    return _exists

In [201]:
def get_existing_solution_version(recipe_arn, name):
    _solution_version_arn = ''
    _solution_arn = 'arn:aws:personalize:{}:{}:solution/{}'.format(region, account_num,
                                                                   name)
    _status = 'UNKNOWN'
    
    try:
        _resp = personalize.describe_solution(solutionArn = _solution_arn)
        _latest_version = _resp['solution']['latestSolutionVersion']
        _status = _latest_version['status']
        if _status in ['CREATE IN_PROGRESS','ACTIVE']:
            _solution_version_arn = _latest_version['solutionVersionArn']
    except Exception as e:
        pass
        
    return _solution_version_arn, _status, _solution_arn

In [190]:
def get_existing_campaign(recipe_arn, name):
    _solution_version_arn = ''
    _solution_arn = 'arn:aws:personalize:{}:{}:solution/{}'.format(region, account_num,
                                                                   name)
    _campaign_arn = 'arn:aws:personalize:{}:{}:campaign/{}'.format(region, account_num,
                                                                   name)
    _campaign_status = 'UNKNOWN'
    
    try:
        _resp = personalize.describe_campaign(campaignArn = _campaign_arn)
        _campaign_status = _resp['campaign']['status']
        if _campaign_status in ['CREATE_IN_PROGESS','ACTIVE']:
            _campaign_arn = _resp['campaign']['campaignArn']
            _solution_version_arn = _resp['campaign']['solutionVersionArn']
    except Exception as e:
        _solution_version_arn = ''
        _campaign_arn = ''
        
    return _solution_version_arn, _solution_arn, _campaign_status, _campaign_arn

In [191]:
def wait_for_solution_version(solution_version_arn, name):
    _latest_time = time.time() + MAX_WAIT_TIME
    _first_time_through = True
    
    while time.time() < _latest_time:
        describe_solution_version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        _status = describe_solution_version_response['solutionVersion']['status']

        if _status in ['ACTIVE', 'CREATE FAILED']:
            if _status == 'CREATE FAILED':
                print('*** Solution version creation failed ***')
            break

        print('SolutionVersion: {} - {}...'.format(name, _status))
        time.sleep(SLEEP_TIME)
    
    _get_solution_metrics_response = personalize.get_solution_metrics(
        solutionVersionArn = solution_version_arn
    )

    print(json.dumps(_get_solution_metrics_response, indent=2))    

In [202]:
def make_campaign(recipe_arn, name, dataset_group_arn):
    print('Entered make_campaign for {}'.format(name))
    
    # if a campaign is already active or being created, don't bother creating it
    (_solution_version_arn, _solution_arn, _campaign_status, _campaign_arn) = \
        get_existing_campaign(recipe_arn, name)
    if _campaign_status in ['CREATE IN_PROGESS','ACTIVE']:
        print('campaign already exists: {}, {}'.format(_campaign_status, _campaign_arn))
        return _solution_version_arn, _solution_arn, _campaign_arn
    
    # otherwise, create a solution if needed
    if  not solution_exists(_solution_arn):
        print('Creating new solution for {}...'.format(name))
        create_solution_response = personalize.create_solution(
            name = name,
            datasetGroupArn = dataset_group_arn,
            recipeArn       = recipe_arn
        )
        _solution_arn = create_solution_response['solutionArn']
        time.sleep(20)
        print('created solution: {}'.format(_solution_arn))

    if _solution_version_arn != '':
        print('solution version already ACTIVE: {}'.format(_solution_version_arn))
    else:
        (_solution_version_arn, _status, _solution_arn) = \
            get_existing_solution_version(recipe_arn, name)
        if _status in ['CREATE IN_PROGRESS','ACTIVE']:
            print('Solution version arn: {}'.format(_solution_version_arn))
        else:
            print('Creating a new solution version for solution: {}...'.format(_solution_arn))
            create_solution_version_response = personalize.create_solution_version(
                solutionArn = _solution_arn
            )
            _solution_version_arn = create_solution_version_response['solutionVersionArn']

    wait_for_solution_version(_solution_version_arn, name)
    create_campaign_response = personalize.create_campaign(
        name = name,
        solutionVersionArn = _solution_version_arn,
        minProvisionedTPS = 1
    )
    _campaign_arn = create_campaign_response['campaignArn']
    print('Waiting for campaign to become active...')

    latest_time = time.time() + MAX_WAIT_TIME
    while time.time() < latest_time:
        describe_campaign_response = personalize.describe_campaign(
            campaignArn = _campaign_arn
        )
        status = describe_campaign_response['campaign']['status']
        print('Campaign: {} - {}'.format(name, status))

        if status == 'ACTIVE' or status == 'CREATE FAILED':
            break

        time.sleep(SLEEP_TIME)
        
    print('Exiting make_campaign for {}'.format(name))
    return(_solution_arn, _solution_version_arn, _campaign_arn)

## Create multiple campaigns in parallel
Creating solutions and campaigns can be time-consuming depending on the size and
complexity of the datasets and the chosen recipe. Here we create a set of campaigns
in parallel. In practice, a given application may only need one or two campaigns.
However, in development and testing, you may want to try out a full suite of recipes
to find out how well they serve your specific requirements and data.

In [203]:
from multiprocessing import Process

def make_campaigns_in_parallel(campaigns, dg_arn):
    jobs = []
    for i in campaigns:
        p = Process(target = make_campaign, args=(i[0], i[1], dg_arn))
        jobs.append(p)
        
    for p in jobs:
        p.start()
        
    for p in jobs:
        p.join()

In [204]:
%%time
campaigns = [['arn:aws:personalize:::recipe/aws-personalized-ranking', 'car-personalized-ranking'],
             ['arn:aws:personalize:::recipe/aws-sims',                 'car-sims'],
             ['arn:aws:personalize:::recipe/aws-popularity-count',     'car-popularity-count'],
             ['arn:aws:personalize:::recipe/aws-hrnn-metadata',        'car-hrnn-metadata'],
             ['arn:aws:personalize:::recipe/aws-hrnn',                 'car-hrnn']
            ]
print(dataset_group_arn)
make_campaigns_in_parallel(campaigns, dataset_group_arn)

arn:aws:personalize:us-east-1:355151823911:dataset-group/car-dg12
Entered make_campaign for car-personalized-ranking
Entered make_campaign for car-sims
Entered make_campaign for car-popularity-count
Entered make_campaign for car-hrnn-metadata
Entered make_campaign for car-hrnn
_solution_arn: arn:aws:personalize:us-east-1:355151823911:solution/car-hrnn
_solution_arn: arn:aws:personalize:us-east-1:355151823911:solution/car-popularity-count
_solution_arn: arn:aws:personalize:us-east-1:355151823911:solution/car-sims
_solution_arn: arn:aws:personalize:us-east-1:355151823911:solution/car-hrnn-metadata
_solution_arn: arn:aws:personalize:us-east-1:355151823911:solution/car-personalized-ranking
Creating a new solution version for solution: arn:aws:personalize:us-east-1:355151823911:solution/car-hrnn...
Creating a new solution version for solution: arn:aws:personalize:us-east-1:355151823911:solution/car-popularity-count...
Creating a new solution version for solution: arn:aws:personalize:us-east

SolutionVersion: car-popularity-count - CREATE IN_PROGRESS...
SolutionVersion: car-sims - CREATE IN_PROGRESS...
SolutionVersion: car-personalized-ranking - CREATE IN_PROGRESS...
SolutionVersion: car-hrnn-metadata - CREATE IN_PROGRESS...
SolutionVersion: car-popularity-count - CREATE IN_PROGRESS...
SolutionVersion: car-hrnn - CREATE IN_PROGRESS...
SolutionVersion: car-hrnn-metadata - CREATE IN_PROGRESS...
SolutionVersion: car-sims - CREATE IN_PROGRESS...
SolutionVersion: car-personalized-ranking - CREATE IN_PROGRESS...
SolutionVersion: car-hrnn - CREATE IN_PROGRESS...
SolutionVersion: car-popularity-count - CREATE IN_PROGRESS...
SolutionVersion: car-hrnn-metadata - CREATE IN_PROGRESS...
SolutionVersion: car-sims - CREATE IN_PROGRESS...
SolutionVersion: car-personalized-ranking - CREATE IN_PROGRESS...
SolutionVersion: car-popularity-count - CREATE IN_PROGRESS...
SolutionVersion: car-hrnn - CREATE IN_PROGRESS...
SolutionVersion: car-hrnn-metadata - CREATE IN_PROGRESS...
SolutionVersion: c

}
Waiting for campaign to become active...
Campaign: car-personalized-ranking - CREATE PENDING
Campaign: car-sims - CREATE IN_PROGRESS
Campaign: car-hrnn - CREATE IN_PROGRESS
Campaign: car-personalized-ranking - CREATE IN_PROGRESS
Campaign: car-sims - CREATE IN_PROGRESS
{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:355151823911:solution/car-hrnn-metadata/175c1b8d",
  "metrics": {
    "coverage": 0.0156,
    "mean_reciprocal_rank_at_25": 0.0049,
    "normalized_discounted_cumulative_gain_at_10": 0.0067,
    "normalized_discounted_cumulative_gain_at_25": 0.0111,
    "normalized_discounted_cumulative_gain_at_5": 0.0043,
    "precision_at_10": 0.0013,
    "precision_at_25": 0.0012,
    "precision_at_5": 0.0011
  },
  "ResponseMetadata": {
    "RequestId": "0dba0554-2b76-4131-a67e-936e372bdf91",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jul 2019 01:49:48 GMT",
      "x-amzn-requestid": "0dba0554-2b76

In [155]:
#make_campaign('arn:aws:personalize:::recipe/aws-hrnn', 'car-hrnn', dataset_group_arn)

In [156]:
#make_campaign('arn:aws:personalize:::recipe/aws-sims', 'car-sims', dataset_group_arn)

In [157]:
#make_campaign('arn:aws:personalize:::recipe/aws-personalized-ranking', 'car-personalized-ranking', dataset_group_arn)

In [158]:
#make_campaign('arn:aws:personalize:::recipe/aws-hrnn-metadata', 'car-hrnn-metadata', dataset_group_arn)

In [211]:
def display_solution_metrics(campaigns):
    for c in campaigns:
        (_solution_version_arn, _status, _solution_arn) = \
            get_existing_solution_version(c[0], c[1])
        if _status != 'ACTIVE':
            print('Solution version for {} does not exist'.format(c[1]))
        else:
            _get_solution_metrics_response = personalize.get_solution_metrics(
                solutionVersionArn = _solution_version_arn
            )

            print('\n{}\n{}'.format(c[1],
                                    json.dumps(_get_solution_metrics_response['metrics'], indent=2))) 

In [212]:
display_solution_metrics(campaigns)


car-personalized-ranking
{
  "coverage": 0.0008,
  "mean_reciprocal_rank_at_25": 0.001,
  "normalized_discounted_cumulative_gain_at_10": 0.0017,
  "normalized_discounted_cumulative_gain_at_25": 0.0017,
  "normalized_discounted_cumulative_gain_at_5": 0.0012,
  "precision_at_10": 0.0003,
  "precision_at_25": 0.0001,
  "precision_at_5": 0.0003
}

car-sims
{
  "coverage": 0.552,
  "mean_reciprocal_rank_at_25": 0.015,
  "normalized_discounted_cumulative_gain_at_10": 0.0211,
  "normalized_discounted_cumulative_gain_at_25": 0.028,
  "normalized_discounted_cumulative_gain_at_5": 0.0151,
  "precision_at_10": 0.0036,
  "precision_at_25": 0.0026,
  "precision_at_5": 0.0038
}

car-popularity-count
{
  "coverage": 0.0008,
  "mean_reciprocal_rank_at_25": 0.0003,
  "normalized_discounted_cumulative_gain_at_10": 0.0002,
  "normalized_discounted_cumulative_gain_at_25": 0.001,
  "normalized_discounted_cumulative_gain_at_5": 0.0,
  "precision_at_10": 0.0,
  "precision_at_25": 0.0002,
  "precision_at_5":